In [1]:
from google.colab import drive
drive.mount('/content/drive')

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Parameters
batch_size = 16
img_height = 224
img_width = 224
data_dir = '/content/drive/MyDrive/posttnatal'

Mounted at /content/drive


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG19, ResNet50, DenseNet121, MobileNetV2
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Load and prepare dataset
batch_size = 16
img_height = 224
img_width = 224
data_dir = '/content/drive/MyDrive/postnatal'

dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    seed=123
)
class_names = dataset.class_names
print(f"Class names: {class_names}")

# Split dataset into train, val, test
train_size = 0.8
val_size = 0.1

total_count = len(dataset)
train_count = int(total_count * train_size)
val_count = int(total_count * val_size)

train_ds = dataset.take(train_count)
val_ds = dataset.skip(train_count).take(val_count)
test_ds = dataset.skip(train_count + val_count)

# Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.1),
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

# Function to create a model
def create_model(base_model, input_shape, num_classes):
    base_model.trainable = False
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create individual models
vgg19_base = VGG19(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
resnet50_base = ResNet50(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
densenet121_base = DenseNet121(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
mobilenetv2_base = MobileNetV2(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')

vgg19_model = create_model(vgg19_base, (img_height, img_width, 3), len(class_names))
resnet50_model = create_model(resnet50_base, (img_height, img_width, 3), len(class_names))
densenet121_model = create_model(densenet121_base, (img_height, img_width, 3), len(class_names))
mobilenetv2_model = create_model(mobilenetv2_base, (img_height, img_width, 3), len(class_names))

# Train all models together for 10 epochs
epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    vgg19_model.fit(train_ds, validation_data=val_ds, epochs=1, verbose=1)
    resnet50_model.fit(train_ds, validation_data=val_ds, epochs=1, verbose=1)
    densenet121_model.fit(train_ds, validation_data=val_ds, epochs=1, verbose=1)
    mobilenetv2_model.fit(train_ds, validation_data=val_ds, epochs=1, verbose=1)

# Evaluate models
vgg19_acc = vgg19_model.evaluate(test_ds)[1]
resnet50_acc = resnet50_model.evaluate(test_ds)[1]
densenet121_acc = densenet121_model.evaluate(test_ds)[1]
mobilenetv2_acc = mobilenetv2_model.evaluate(test_ds)[1]

print(f"VGG19 Test Accuracy: {vgg19_acc:.2f}")
print(f"ResNet50 Test Accuracy: {resnet50_acc:.2f}")
print(f"DenseNet121 Test Accuracy: {densenet121_acc:.2f}")
print(f"MobileNetV2 Test Accuracy: {mobilenetv2_acc:.2f}")

# Combine predictions (Ensemble)
def ensemble_predictions(models, test_ds):
    all_preds = []
    all_true = []

    for images, labels in test_ds:
        all_true.extend(labels.numpy())
        model_preds = np.zeros((images.shape[0], len(class_names)))

        # Collect predictions from each model
        for model in models:
            preds = model.predict(images)
            model_preds += preds  # Sum the probabilities

        # Average the predictions (for soft-voting)
        avg_preds = model_preds / len(models)
        all_preds.extend(np.argmax(avg_preds, axis=1))

    return np.array(all_true), np.array(all_preds)

# Ensemble models
models = [vgg19_model, resnet50_model, densenet121_model, mobilenetv2_model]
y_true, y_pred = ensemble_predictions(models, test_ds)

# Classification report
print("\nEnsemble Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

Found 2999 files belonging to 2 classes.
Class names: ['downSyndorme', 'healty']
Epoch 1/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 346s 2s/step - accuracy: 0.5875 - loss: 1.2415 - val_accuracy: 0.7118 - val_loss: 0.5539
150/150 ━━━━━━━━━━━━━━━━━━━━ 71s 399ms/step - accuracy: 0.6883 - loss: 0.6781 - val_accuracy: 0.7153 - val_loss: 0.5381
150/150 ━━━━━━━━━━━━━━━━━━━━ 86s 410ms/step - accuracy: 0.5938 - loss: 1.4589 - val_accuracy: 0.5833 - val_loss: 0.6397
150/150 ━━━━━━━━━━━━━━━━━━━━ 70s 401ms/step - accuracy: 0.5485 - loss: 0.8191 - val_accuracy: 0.5972 - val_loss: 0.6553
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 54s 361ms/step - accuracy: 0.6917 - loss: 0.5727 - val_accuracy: 0.7049 - val_loss: 0.5726
150/150 ━━━━━━━━━━━━━━━━━━━━ 50s 331ms/step - accuracy: 0.7671 - loss: 0.4815 - val_accuracy: 0.7639 - val_loss: 0.4968
150/150 ━━━━━━━━━━━━━━━━━━━━ 53s 352ms/step - accuracy: 0.6610 - loss: 0.6303 - val_accuracy: 0.6979 - val_loss: 0.6103
150/150 ━━━━━━━━━━━━━━━━━━━━ 48s 321ms/step - accuracy: 0.6

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

Ensemble Classification Report:
              precision    recall  f1-score   support

downSyndorme       0.76      0.92      0.84       158
      healty       0.90      0.71      0.79       153

    accuracy                           0.82       311
   macro avg       0.83      0.81      0.81       311
weighted avg       0.83      0.82      0.81       311

